In [1]:
## Load packages

current best test score 0.9203326182684942
That is with the entire kitchen sink thrown at it.
but with squarefoot not combined

current best test score 0.9212150618551485

train, test = add_price_comp_feature(train, test,'Neighborhood')
train, test = add_price_comp_feature(train, test,'GarageCars')
train, test = add_price_comp_feature(train, test,'BldgType')
train, test = add_price_comp_feature(train, test,'MSZoning')
train, test = add_price_comp_feature(train, test,'Condition1')

smaller_num_feat_test = [
    'OverallQual',
    'overall_score',
    'total_sf',
    'GrLivArea',
    'year_since_built',
    'LotArea',
    'GarageArea',
    'year_since_remod',
    'BsmtExposure',
    'KitchenQual'
    
]

current best test score: 0.9235239696528883

this is with the absolute kitchen sink including every the squarefoot combo stuff

current best test score: 0.9230459482712876

this is with the absolute kitchen sink including every the squarefoot combo stuff and now the parks informations

In [2]:
import pandas as pd
import numpy as np
import helper
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.style as style
style.use('fivethirtyeight')

from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
# from sklearn.feature_selection import SelectFromModel

In [3]:
colors = ["#FF0B04", "#F1BE48",
           "#B9975B", "#8B5B29",
           "#524727",
         ]
sns.set_palette(sns.color_palette(colors))

In [4]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 200)

In [5]:
housing = pd.read_csv('Ames_Housing_Price_Data.csv', index_col=0,low_memory = False)


train_, test_ = helper.data_processing_wrapper(housing,
                                               num_to_cat_list = ['MSSubClass','MoSold'],
                                             remove_PID = False
                                        )





In [6]:
parks = pd.read_csv('parksFeatures.csv',index_col = 0)

In [7]:
train = train_.merge(parks, how = 'left', left_on = 'PID', right_on = 'PID')
test = test_.merge(parks, how = 'left', left_on = 'PID', right_on = 'PID')



train = train.dropna(subset=['Ada Hayden'])
train = train.reset_index(drop=True)

test = test.dropna(subset=['Ada Hayden'])
test = test.reset_index(drop=True)

In [8]:
cat_feats = train.select_dtypes(['object','bool']).columns.to_list()
cat_feats


['MSSubClass',
 'MSZoning',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'Foundation',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'CentralAir',
 'Electrical',
 'Functional',
 'GarageType',
 'Fence',
 'MiscFeature',
 'MoSold',
 'SaleType',
 'SaleCondition']

In [9]:
num_cols = train.select_dtypes(['float64','int64']).columns.to_list()
num_cols.remove('SalePrice')

In [10]:
num_cols

['PID',
 'GrLivArea',
 'LotFrontage',
 'LotArea',
 'Alley',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'MasVnrArea',
 'ExterQual',
 'ExterCond',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 'HeatingQC',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'KitchenQual',
 'TotRmsAbvGrd',
 'Fireplaces',
 'FireplaceQu',
 'GarageYrBlt',
 'GarageFinish',
 'GarageCars',
 'GarageArea',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'PoolQC',
 'MiscVal',
 'YrSold',
 'Ada Hayden',
 'Ames Dog Park',
 'band shell park',
 'Moore Memorial',
 'brookside park',
 'emma mccarthy lee park',
 'river valley park',
 'charles & June Calhoun Park',
 'christofferson park',
 'christopher gartner park',
 'country gables park',
 'daley park & greenbelt',
 'duff

define added feature functions

In [11]:
def add_year_since_feature(df):
    df['year_since_built'] = df['YrSold']-df['YearBuilt']
    df['year_since_remod'] = df['YrSold']-df['YearRemodAdd']
    df['year_since_garage'] = df['YrSold']-df['GarageYrBlt']

    df.loc[df['year_since_built']<0,'year_since_built']=0
    df.loc[df['year_since_remod']<0,'year_since_remod']=0
    df.loc[df['year_since_garage']<0,'year_since_garage']=0
    return df

In [12]:
def add_combined_related_num_features(df):
    df['total_sf'] = df['GrLivArea'] + df['TotalBsmtSF']
    df['total_high_qual_finished_sf'] = (df['GrLivArea'] + df['TotalBsmtSF'] - # Adding total square foot.
                                         df['BsmtUnfSF'] - df['LowQualFinSF']) # Subtracting unfinished/low qual
    df['total_deck_sf'] = (df['WoodDeckSF'] + df['OpenPorchSF'] + df['EnclosedPorch'] + 
                           df['3SsnPorch'] + df['ScreenPorch'])
    df['total_full_bath'] = df['BsmtFullBath'] + df['FullBath']
    df['total_half_bath'] = df['BsmtHalfBath'] + df['HalfBath']

    return df

In [13]:
def add_score_feature(df):
    df['overall_score'] = df['OverallQual']*df['OverallCond']
    df['exter_score'] = df['ExterQual']*df['ExterCond']
    df['bsmt_score'] = df['BsmtQual']*df['BsmtCond']
    df['garage_score'] = df['GarageQual']*df['GarageCond']
    return df


In [14]:
def add_non_linear_transformed_features(df,cols):
    df_list = [df]
    for col in cols:
        df_new = pd.DataFrame()
        df_new[col+'_squared'] = df[col]**2
        df_new[col+'_cubed'] = df[col]**3
        df_new[col+'_square_root'] = df[col]**0.5
        df_list.append(df_new)
    df = pd.concat(df_list, axis=1)
    return df

In [15]:
def add_price_comp_feature(train_, test_,comp_feature):
    temp = train.groupby(comp_feature).agg({'SalePrice':'median'})
    temp.columns = [comp_feature+'_comp']
    train_ = train_.merge(temp, how='left', on=comp_feature)
    test_ = test_.merge(temp, how='left', on=comp_feature)
    return train_, test_


In [16]:
def add_price_comp_log_feature(train_, test_,comp_feature):
    temp = train_.copy()
    temp['log_SalePrice'] = np.log(temp['SalePrice'])
    temp = temp.groupby(comp_feature).agg({'log_SalePrice':'median'})
    temp.columns = [comp_feature+'_log_comp']
    train_ = train_.merge(temp, how='left', on=comp_feature)
    test_ = test_.merge(temp, how='left', on=comp_feature)
    return train_, test_

In [17]:
def lasso_grid_cv(train_,test_,cat_feats_,
                  starting_alphas_=[0.0001, 0.0003, 0.0006, 0.001, 
                                    0.003, 0.006, 0.01, 0.03, 0.06, 0.1,
                                    0.3, 0.6, 1],
                  n_jobs_ = None,
                  cv_ = 5
                  
                 ):

    scaler = StandardScaler(with_mean=False)
    lasso = Lasso(max_iter = 50000, )

    X = train_.drop(['SalePrice'],axis=1)
    transformer = ColumnTransformer([("Cat", 
                                      OneHotEncoder(handle_unknown = 'ignore'), 
                                      cat_feats_)], remainder='passthrough')
    X = transformer.fit_transform(X)
    X = scaler.fit_transform(X)
    y = np.log(train['SalePrice'])

    # Grid Search set up.

    alphas = [0.0001, 0.0003, 0.0006, 0.001, 0.003, 0.006, 0.01, 0.03, 0.06, 0.1, 
                              0.3, 0.6, 1]

    tuned_parameters = [{'alpha': alphas}]
    print(f'Performing Grid Search with alphas of: {alphas}')
    clf = GridSearchCV(lasso, tuned_parameters, cv=cv_,n_jobs = n_jobs_)
    clf.fit(X, y)

    # best alpha with first draft. Now iterate for more precision with alphas.
    best_alpha = clf.best_params_['alpha']
    best_score = clf.best_score_
    print(f'Current best alpha: {best_alpha}')
    print(f'Current best CV R2: {best_score}')
    best_score_last = 1
    best_score_diff = abs(best_score-best_score_last)
    while best_score_diff > .001:
        best_score_last = best_score
        alphas_multiply = np.array([.3,.4,.5,.6,.7,.8,.9,1,
                            1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9])
        alphas = alphas_multiply*best_alpha
        tuned_parameters = [{'alpha': alphas}]
        print(f'Performing Grid Search with alphas of: {alphas}')
        clf = GridSearchCV(lasso, tuned_parameters, cv=5)
        clf.fit(X, y)
        best_alpha = clf.best_params_['alpha']
        best_score = clf.best_score_
        
        print(f'Current best alpha: {best_alpha}')
        print(f'Current best CV R2: {best_score}')        
        best_score_diff = abs(best_score-best_score_last)
    print('Modeling complete :)')
    return clf, transformer, scaler
    







In [18]:
train, test = add_price_comp_feature(train, test,'Neighborhood')
train, test = add_price_comp_feature(train, test,'GarageCars')
train, test = add_price_comp_feature(train, test,'BldgType')
train, test = add_price_comp_feature(train, test,'MSZoning')

##
# train, test = add_price_comp_feature(train, test,'HouseStyle')
# train, test = add_price_comp_feature(train, test,'FullBath')
# train, test = add_price_comp_feature(train, test,'MSSubClass')
# train, test = add_price_comp_feature(train, test,'LotShape')
# train, test = add_price_comp_feature(train, test,'LotConfig')

##
train, test = add_price_comp_feature(train, test,'Condition1')

##
# train, test = add_price_comp_feature(train, test,'MasVnrType')
# train, test = add_price_comp_feature(train, test,'Foundation')
# train, test = add_price_comp_feature(train, test,'BsmtFinType1')
# train, test = add_price_comp_feature(train, test,'GarageType')
# train, test = add_price_comp_feature(train, test,'Fence')


train, test = add_price_comp_log_feature(train, test,'Neighborhood')
train, test = add_price_comp_log_feature(train, test,'GarageCars')
train, test = add_price_comp_log_feature(train, test,'BldgType')
train, test = add_price_comp_log_feature(train, test,'MSZoning')

##
# train, test = add_price_comp_log_feature(train, test,'HouseStyle')
# train, test = add_price_comp_log_feature(train, test,'FullBath')
# train, test = add_price_comp_log_feature(train, test,'MSSubClass')
# train, test = add_price_comp_log_feature(train, test,'LotShape')
# train, test = add_price_comp_log_feature(train, test,'LotConfig')

###
train, test = add_price_comp_log_feature(train, test,'Condition1')


# 
# train, test = add_price_comp_log_feature(train, test,'MasVnrType')
# train, test = add_price_comp_log_feature(train, test,'Foundation')
# train, test = add_price_comp_log_feature(train, test,'BsmtFinType1')
# train, test = add_price_comp_log_feature(train, test,'GarageType')
# train, test = add_price_comp_log_feature(train, test,'Fence')






In [19]:
smaller_num_feat_test = [
    'OverallQual',
    'overall_score',
    'total_sf',
    'GrLivArea',
    'year_since_built',
    'LotArea',
    'GarageArea',
    'year_since_remod',
    'BsmtExposure',
    'KitchenQual'
    
]

In [20]:
train = add_year_since_feature(train)
train = add_score_feature(train)
train = add_combined_related_num_features(train)

num_cols = train.select_dtypes(['float64','int64']).columns.to_list()
num_cols.remove('SalePrice')
train = add_non_linear_transformed_features(train,smaller_num_feat_test)
# train = add_non_linear_transformed_features(train,num_cols)



In [21]:
test = add_year_since_feature(test)
test = add_score_feature(test)
test = add_combined_related_num_features(test)
test = add_non_linear_transformed_features(test,smaller_num_feat_test)

# test = add_non_linear_transformed_features(test,num_cols)



In [22]:
# train, test = add_price_comp_feature(train, test,'overall_score')

In [23]:
num_cols = train.select_dtypes(['float64','int64']).columns.to_list()
num_cols.remove('SalePrice')

In [24]:
clf, transformer, scaler = lasso_grid_cv(train,test,cat_feats,n_jobs_ = -1)

Performing Grid Search with alphas of: [0.0001, 0.0003, 0.0006, 0.001, 0.003, 0.006, 0.01, 0.03, 0.06, 0.1, 0.3, 0.6, 1]
Current best alpha: 0.001
Current best CV R2: 0.946377746839054
Performing Grid Search with alphas of: [0.0003 0.0004 0.0005 0.0006 0.0007 0.0008 0.0009 0.001  0.0011 0.0012
 0.0013 0.0014 0.0015 0.0016 0.0017 0.0018 0.0019]
Current best alpha: 0.0013000000000000002
Current best CV R2: 0.9465061105725935
Modeling complete :)


In [25]:
clf.best_score_

0.9465061105725935

In [26]:
X_tst = test.drop(['SalePrice'],axis=1)
X_tst = transformer.transform(X_tst)
X_tst = scaler.transform(X_tst)
y_tst = np.log(test['SalePrice'])

clf.score(X_tst,y_tst)

0.9204376273650542

In [27]:
columns_transformed = transformer.named_transformers_['Cat'].get_feature_names(input_features= cat_feats)
new_columns = list(columns_transformed)+num_cols

coef_df = pd.DataFrame({'features':new_columns,'coefs':clf.best_estimator_.coef_})
coef_df = coef_df[coef_df['coefs']!=0]
coef_df['coefs_abs'] = abs(coef_df['coefs'])
coef_df = coef_df.sort_values('coefs_abs',ascending=False).reset_index(drop=True)
coef_df


,features,coefs,coefs_abs
0,year_since_built_square_root,-6.486408e-02,6.486408e-02
1,GrLivArea_square_root,6.324315e-02,6.324315e-02
2,total_sf_square_root,6.224721e-02,6.224721e-02
3,overall_score_square_root,5.409827e-02,5.409827e-02
4,total_high_qual_finished_sf,3.729250e-02,3.729250e-02
5,LotArea_square_root,3.663709e-02,3.663709e-02
6,OverallQual_cubed,2.975652e-02,2.975652e-02
7,Neighborhood_comp,2.506808e-02,2.506808e-02
8,GarageCars,1.447157e-02,1.447157e-02
9,MSZoning_log_comp,1.389790e-02,1.389790e-02


moore park
DistToPlayground
DistToPark
DistToRec
DistToPicnic
halfMileParks

drop PID

In [31]:
len(list(parks.columns))

40